In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
catalog.list()

In [ ]:
transactions = context.catalog.load('transactions_sample')

In [ ]:
customers = context.catalog.load('customers_sample')

In [ ]:
articles = context.catalog.load('articles_sample')

In [ ]:
candidates = context.catalog.load('candidates')

In [ ]:
candidates.shape

In [ ]:
candidates[candidates.isna().any(axis=1)]

In [ ]:
transactions[transactions.customer_id.isin(['ff18c106287931d170618b59fd402ed0ecc292869aad5f'])]

In [ ]:
transactions[transactions.customer_id.isin(candidates[candidates.isna().any(axis=1)].customer_id.to_list())]

In [ ]:
transactions[~(transactions.customer_id.isin(candidates[candidates.isna().any(axis=1)].customer_id.to_list()))].shape

In [ ]:
transactions.shape, customers.shape, articles.shape

In [ ]:
my_dict = {'01': {'0101', '02020', '03030'}, '02': {'02032', '32131', '321321', '32131321'}}

In [ ]:
my_dict

In [ ]:
my_df = pd.DataFrame(my_dict.keys(), columns=['blasda'])

In [ ]:
my_df

In [ ]:
my_df['blabla'] = my_df.loc[:, 'blasda'].map(my_dict)

In [ ]:
my_df

In [ ]:
pd.DataFrame.from_dict(my_dict, orient='index')

### Popularity items

- recent popular items (1, 3, 7, 30, 90 days), most popular products (overall)
- products purchased previously (all)
- popular items by SEGMENTS (age, gender) (1, 3, 7, 30, 90 days)
- items which have the same `prod_name`
<br><br>
- IMAGE/TEXT EMBEDDINGS
- BASKET MARKET ANALYSIS
<br><br>
- just choose the `N` that maximizes recall for EACH METHOD

In [ ]:
transactions.t_dat = pd.to_datetime(transactions.t_dat)

In [ ]:
transactions.t_dat.describe(datetime_is_numeric=True)[['min', 'max']]

In [ ]:
transactions.head()

## global articles

In [ ]:
articles_bought_by_most_customers = (transactions
    .groupby(['article_id'])['customer_id']
    .nunique()
    .reset_index(name='cd_customer_id')
    .sort_values(by='cd_customer_id', ascending=False)
)

In [ ]:
articles_bought_by_most_customers.head(200).article_id.to_list()

In [ ]:
articles_sold_most = (transactions
    .groupby(['article_id'])['customer_id']
    .size()
    .reset_index(name='sales_sum_count')
    .sort_values(by='sales_sum_count', ascending=False)
)

In [ ]:
articles_sold_most.head()

### trending articles (nope)

Trending articles seem to much engineered by hand, you can just take popularity items from last 7 days. I suspect result will be comparable

In [ ]:
tmp = transactions[['t_dat']].copy()

In [ ]:
tmp['day_of_week'] = tmp['t_dat'].dt.dayofweek

In [ ]:
tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['day_of_week'] - 1, unit='D')

In [ ]:
tmp.loc[tmp['day_of_week'] >=2 , 'ldbw'] = tmp.loc[tmp['day_of_week'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['day_of_week'] >=2])) * 7, unit='D')

ldbw - next nearest last day (in our case thursday) or current

## segment articles

In [ ]:
transactions.head()

In [ ]:
customers.head()

In [ ]:
customers['age_bin'] = pd.qcut(customers['age'], q=5, labels=False).astype(str).str.zfill(2)

In [ ]:
cols = ['age']

In [ ]:
customers.groupby(['age_bin'])['age'].agg(['min', 'max'])

- first question -> what about age_bin (WHERE to do it)
- second -> filling NA -> it's done in SAMPLING, but nowhere else

In [ ]:
articles_sold_most_by_segment = (transactions
    .merge(customers, on='customer_id')
    .groupby(['age_bin', 'article_id'])['customer_id']
    .size()
    .reset_index(name='most_sold_by_segment')
    .sort_values(by='most_sold_by_segment', ascending=False)
)

In [ ]:
articles_sold_most_by_segment.query('age_bin=="1"')

In [ ]:
articles_sold_most_by_segment.query('age_bin=="5"')

check how much they intersect, maybe there is no point in doing for different days?

## Previously bought

### previously bought the same `prod_name`

In [ ]:
customers_prod_name = (
    transactions
    .merge(articles[['article_id', 'prod_name']], on='article_id')[['customer_id', 'prod_name']]
    .drop_duplicates()
    .sort_values(by='customer_id')
)

In [ ]:
customers_prod_name.shape

In [ ]:
prev_bought_prod_name = (
    customers_prod_name
    .merge(articles[['article_id', 'prod_name']], on='prod_name')
    .drop(['prod_name'], axis=1)
)

In [ ]:
prev_bought_prod_name.shape

### previously bought

In [ ]:
prev_bought = transactions[['customer_id', 'article_id']].drop_duplicates()

In [ ]:
prev_bought.groupby(['customer_id'])['article_id'].apply(list).reset_index(name='previously_bought')

### embeddings search - faiss

In [ ]:
emb = context.catalog.load('text_embeddings')

In [ ]:
emb

In [ ]:
from sklearn.neighbors import KDTree

In [ ]:
tree = KDTree(emb.values, leaf_size=5)

In [ ]:
def find_similar_images(query_article_id, embeddings, tree):
    _, ind = tree.query(embeddings.loc[query_article_id].values.reshape(1, -1), k=5)
    closest_embeddings = embeddings.iloc[ind[0]].index.tolist()
    return closest_embeddings

In [ ]:
find_similar_images('0108775015', emb, tree)

In [ ]:
emb.loc[['0108775051', '0108775044', '0108775015', '0623522001', '0623522010'], :]

In [ ]:
transactions = context.catalog.load('transactions_sample')

In [ ]:
transactions.sort_values(by='t_dat', ascending=False, inplace=True)

In [ ]:
sample = transactions.head(100)

In [ ]:
sample['article_id'].apply(lambda x: find_similar_images(x, emb, tree))

In [ ]:
transactions

In [ ]:
transactions.groupby(['customer_id']).head(5)

In [ ]:
transactions.groupby(['customer_id'])['article_id'].count().reset_index().groupby(['article_id'])['customer_id'].nunique().to_clipboard()

correct sum

In [ ]:
candidates = context.catalog.load('candidates')

In [ ]:
candidates.shape

In [ ]:
candidates.customer_id.nunique()

In [ ]:
candidates

In [ ]:
my_list = ['321312', '123', '3213']

In [ ]:
[my_list for i in candidates.index]

In [ ]:
pd.Series(my_list * 8)

In [ ]:
all_customers.loc[:, 'global_articles'] = pd.Series(list(articles) * len(all_customers))